# First Project Pipline

## Setup

In [2]:
import spacy
from spacy.tokens import Token
from spacy.tokens import Doc
from spacy.language import Language

In [3]:
from gensim.summarization import keywords

In [4]:
from wordfreq import zipf_frequency
from wordfreq import word_frequency

In [5]:
import numpy as np
import pandas as pd
import regex as re

In [6]:
import nltk
#nltk.download('cmudict')

In [7]:
from nltk.corpus import cmudict

phoneme_dict = dict(cmudict.entries())

def syllable_counter(word):
    '''function that counts a syllable in a word'''
    if word not in phoneme_dict:
        return 0
    syllables = phoneme_dict[word]
    count = len([syllable for syllable in syllables if syllable[-1].isdigit()])
    return round(count, 0)

In [8]:
class Minimum():
    def __init__(self, initial_value):
        self.value = initial_value
    
    def update_minimum(self, potential_min):
        if potential_min < self.value:
            self.value = potential_min
            
class Maximum():
    def __init__(self, initial_value):
        self.value = initial_value
    
    def update_maximum(self, potential_min):
        if potential_min > self.value:
            self.value = potential_min

In [9]:
!python --version

Python 3.8.0


## Data

In [10]:
text_file = '../data/kafka.txt'

In [11]:
#print(open(text_file, "r").read())

# Pipeline Components

## Preprocessing

### Wordcount

In [12]:
def wordcount(doc):
    '''gives an overall word count'''
    
    for token in doc:
        if token.is_alpha:
            doc._.wordcount += 1
    
    print(f'{doc._.wordcount} words in document')
         
    return doc

### Filter

In [13]:
def filter_tokens(doc):
    '''filters all tokens'''           
    
    
    for token in doc:
        # filter stopwords
        if not token.is_alpha or token.is_stop:
            token._.is_excluded = True
            
        # filter part-of-speech
        elif token.pos_ not in ['NOUN', 'VERB', 'ADJ']:
            token._.is_excluded = True
            
        # filter entities
        elif token.ent_type != 0:
                token._.is_excluded = True
                
        # count included words   
        else:
            doc._.included_wordcount += 1

    print(f'{doc._.included_wordcount} words in vocabulary')
         
    return doc

### Eliminating Duplicates

In [14]:
def elim_dup(doc):
    '''eliminates all duplicates and counts the appearance of the included words'''
    already_appeared = {}

    for token in doc:
        if not token._.is_excluded:
            if token.lemma_ in already_appeared.keys():
                already_appeared[token.lemma_]._.appearance += 1
                token._.is_excluded = True
                doc._.included_wordcount -= 1
            else:
                token._.appearance = 1
                already_appeared[token.lemma_] = token
    
    print(f'{doc._.included_wordcount} words in vocabulary without duplicates')
    
    return doc

## Word Difficulty

### Rating Difficulty

In [15]:
def syl_weight(n):
    w = 0
    for i in range(n):
        w += 0.5**(i+1)
    return w

In [16]:
def get_difficulty(doc):
   
    for token in doc:
        if not token._.is_excluded:
            lemma = token.lemma_
            difficulty = 8 - zipf_frequency(lemma, 'en') # score of 0-8
            difficulty += syl_weight(syllable_counter(lemma)) # now score of 0-9
            token._.difficulty = round(difficulty/9, 3) #normalised to 0-1
            
    return doc

## Word Relevance

### Relative Freqency

In [17]:
def calculate_relativ_freq(doc):
    '''calculating the relativ frequency of a included word'''
    
    calculate_last = []
    min_freq = Minimum(1)
    max_score = Maximum(0)
    
    def calc_rel_freq(word_freq, token):
        return round(((token._.appearance/doc._.wordcount) **2) / word_freq, 3)
    
    for token in doc:
        if not token._.is_excluded:
            overall_word_freq = word_frequency(token.lemma_, 'en')
            
            if overall_word_freq == 0:
                calculate_last.append(token)
            else:
                min_freq.update_minimum(overall_word_freq)
                token._.relativ_freq = calc_rel_freq(overall_word_freq, token)
                max_score.update_maximum(token._.relativ_freq)
    
    for token in calculate_last:
        token._.relativ_freq = calc_rel_freq(min_freq.value, token)
        max_score.update_maximum(token._.relativ_freq)
        
    for token in doc:
        if not token._.is_excluded:
            token._.relativ_freq /= max_score.value
    
    
    return doc

### Keywords Extraction

In [18]:
from yake import KeywordExtractor

In [20]:
def check_keyphrases(doc):
    
    kw_extractor = KeywordExtractor(lan="en", n=1, top=100)
    kw = dict(kw_extractor.extract_keywords(doc.text))
    
    keywords_token = []
    already_in_list = []
    for token in doc:
        if not token._.is_excluded:
            if token.lemma_ in kw.keys():
                token._.keyword_score = 1 - kw[token.lemma_]

    return doc

In [21]:
'''         
        if token.text in kw:
            token._.is_keyword = True
            if token.text not in already_in_list:
                keywords_token.append(token)
                already_in_list.append(token.text)
    doc._.keywords = keywords_token
    
    print(f'{len(keywords_token)} words are keywords')
    
    for token in doc:
        for kw in keywords_token:
            kw_score = token.similarity(kw)
            if kw_score > token._.keyword_score:
                token._.keyword_score =  kw_score'''

"         \n        if token.text in kw:\n            token._.is_keyword = True\n            if token.text not in already_in_list:\n                keywords_token.append(token)\n                already_in_list.append(token.text)\n    doc._.keywords = keywords_token\n    \n    print(f'{len(keywords_token)} words are keywords')\n    \n    for token in doc:\n        for kw in keywords_token:\n            kw_score = token.similarity(kw)\n            if kw_score > token._.keyword_score:\n                token._.keyword_score =  kw_score"

# Creating the Pipeline

In [22]:
def create_pipeline():
    nlp = spacy.load('en_core_web_lg', disable = ['parser'])
    
    ## Preprocessing
    # wordcount
    Doc.set_extension('wordcount', default=0, force=True)
    nlp.add_pipe(wordcount)
    
    # filter tokens
    Doc.set_extension('included_wordcount', default=0, force=True)
    Token.set_extension('is_excluded', default=False, force=True)
    nlp.add_pipe(filter_tokens)
    
    # eliminate dublicates
    Token.set_extension('appearance', default=np.nan, force=True)
    nlp.add_pipe(elim_dup)
    
    
    
    ## Word difficulty
    # difficulty
    Token.set_extension('difficulty', default=0, force=True)
    nlp.add_pipe(get_difficulty)
    
    
    
    ## Word Relevance
    # relative frequency
    Token.set_extension('relativ_freq', default=np.nan, force=True)
    nlp.add_pipe(calculate_relativ_freq)
    
    # keywordscore
    Doc.set_extension('keywords', default=[], force=True)
    Token.set_extension('is_keyword', default=False, force=True)
    Token.set_extension('keyword_score', default=0, force=True)
    nlp.add_pipe(check_keyphrases)
    
    return nlp

In [23]:
nlp = create_pipeline()
nlp.pipe_names

['tagger',
 'ner',
 'wordcount',
 'filter_tokens',
 'elim_dup',
 'get_difficulty',
 'calculate_relativ_freq',
 'check_keyphrases']

# Test Processing

In [24]:
def df_from_doc(doc):
    data = []
    for token in doc:
        if not token._.is_excluded:
            data.append((token, token.lemma_, token._.appearance, token._.difficulty, token._.relativ_freq, token._.keyword_score))
    df = pd.DataFrame(data, columns=['token', 'lemma', 'appearance', 'difficulty', 'relative freqency', 'keyword score'])
    
    df[['difficulty_rank', 'keyword_rank']] = df[['difficulty', 'keyword score']].rank(ascending=False)
    
    df['overall_ranking'] = 3 * df['difficulty'] + df['relative freqency'] + df['keyword score']
    
    return df

In [119]:
def predict_outputsize(df):
    
    def user_knows_vocab(s):
        response = input(f'Can you translate this word:   {s}   [y/n]  ')
        
        while response not in ['y','n']:
            response = input('Unexpected input. Try again.\n' +
                             f'Can you translate this word:   {s}   [y/n]  ')
        return response == 'y'
    
    
    n = len(df)
    dict_word_known = {}
    result = False
    
    i = 1
    loc = 0
    offset = 0
    last = False
    while loc <= n:
        
        print(f'Word position {loc}:')
        if loc in dict_word_known.keys():
            result = dict_word_known[loc]
        else:
            result = user_knows_vocab(df.iloc[loc]['lemma'])
            dict_word_known[loc] = result
        
        if result:
            if last:
                break
            last = True
            offset = last_loc
            i = 0
        else:
            last = False
            
        i += 1 
        last_loc = loc
        loc = (2**i) - 1 + offset
            
    
    return df[:last_loc-1]

In [104]:
predict_outputsize(df)

Word position 0:
Can you translate this word:   deep   [y/n]  n
Word position 3:
Can you translate this word:   structured   [y/n]  n
Word position 7:
Can you translate this word:   method   [y/n]  n
Word position 15:
Can you translate this word:   learn   [y/n]  y
Word position 8:
Can you translate this word:   base   [y/n]  n
Word position 10:
Can you translate this word:   neural   [y/n]  n
Word position 14:
Can you translate this word:   supervised   [y/n]  n
Word position 22:
Can you translate this word:   include   [y/n]  y
Word position 15:


,token,lemma,appearance,difficulty,relative freqency,keyword score,difficulty_rank,keyword_rank,overall_ranking
0,Deep,deep,8,0.390,0.015001,0.955717,91.5,2.0,2.140719
1,learning,learning,7,0.423,0.012882,0.944046,75.0,3.0,2.225928
2,known,know,1,0.266,0.000018,0.000000,110.0,87.5,0.798018
3,structured,structured,2,0.543,0.012652,0.736233,25.0,13.0,2.377885
4,broader,broad,1,0.433,0.000575,0.000000,72.0,87.5,1.299575
5,family,family,1,0.344,0.000051,0.790779,103.5,10.5,1.822830
6,machine,machine,3,0.428,0.002595,0.902369,73.0,5.0,2.188964
7,methods,method,1,0.436,0.000339,0.000000,71.0,87.5,1.308339
8,based,base,1,0.386,0.000213,0.000000,93.0,87.5,1.158213
9,artificial,artificial,1,0.526,0.001449,0.912344,32.0,4.0,2.491793


### Test scentence

In [26]:
doc = nlp("Hi I'm a little little boy. Get me a piece of cake or I'll killed your mother. I am the mother of your mother")

24 words in document
9 words in vocabulary
6 words in vocabulary without duplicates


In [38]:
df = df_from_doc(doc)

In [39]:
df = df.sort_values(by=['overall_ranking'], ascending=False)

In [47]:
df.iloc[:0]

,token,lemma,appearance,difficulty,relative freqency,keyword score,difficulty_rank,keyword_rank,overall_ranking


### Kafka

In [106]:
with open(text_file, "r") as f:
    doc = nlp(f.read())

25062 words in document
7812 words in vocabulary
1873 words in vocabulary without duplicates


In [107]:
df = df_from_doc(doc)

In [108]:
df = df.sort_values(by=['overall_ranking'], ascending=False).head(50)

In [110]:
predict_outputsize(df)

Word position 0:
Can you translate this word:   charwoman   [y/n]  n
Word position 3:
Can you translate this word:   swinge   [y/n]  n
Word position 7:
Can you translate this word:   ebook   [y/n]  y
Word position 4:
Can you translate this word:   teaboy   [y/n]  y


,token,lemma,appearance,difficulty,relative freqency,keyword score,difficulty_rank,keyword_rank,overall_ranking
1335,charwoman,charwoman,8,0.807,1.000000,0.0,7.0,964.0,3.421000
497,fretsaw,fretsaw,1,0.889,0.053115,0.0,3.0,964.0,2.720115
655,soughing,soughing,1,0.889,0.053115,0.0,3.0,964.0,2.720115
396,swinging,swinge,1,0.889,0.053115,0.0,3.0,964.0,2.720115
1358,teaboy,teaboy,1,0.889,0.053115,0.0,3.0,964.0,2.720115
1166,startlement,startlement,1,0.889,0.053115,0.0,3.0,964.0,2.720115


### Wordclusters

In [340]:
doc = nlp("Let's take a ride in my new car. I will drive you home in my car. I love cars, which drive very fast and loud. Trucks and vans are the best. You can also cook in the back of my car. Food, meals and drinks are very tasty. My mother is all in to cooking good meals, foods and healthy dishes. This keeps me healthy while. I love the pasta and pies she bakes")

74 words in document
32 words in vocabulary
22 words in vocabulary without duplicates


In [341]:
df = df_from_doc(doc)

In [342]:
df.sort_values(by=['overall_ranking'], ascending=False).head(50)

,token,lemma,appearance,difficulty,relative freqency,keyword score,word vector,difficulty_rank,keyword_rank,overall_ranking
14,tasty,tasty,1,0.551,1.000000,0.362640,"[-0.5076, -0.32474, 0.44828, 0.18926, 0.017181...",1.0,8.0,3.015640
19,pasta,pasta,1,0.543,0.852215,0.259213,"[-0.47621, -0.12565, 0.57201, -0.41622, 0.0878...",2.0,12.0,2.740427
21,bakes,bake,1,0.519,0.912728,0.000000,"[-0.027228, 0.17126, 0.30655, -0.4484, -0.0939...",3.0,17.5,2.469728
16,healthy,healthy,2,0.441,0.409298,0.656656,"[-0.40781, 0.53036, -0.25297, 0.19766, -0.1103...",8.0,5.0,2.388954
3,car,car,4,0.342,0.375376,0.898871,"[0.20987, 0.46481, -0.24238, -0.065751, 0.6085...",16.5,1.0,2.300247
10,cook,cook,2,0.430,0.577998,0.314925,"[-0.34052, -0.24223, 0.5442, -0.069727, 0.0716...",10.0,9.0,2.182923
12,meals,meal,2,0.448,0.836605,0.000000,"[-0.32314, -0.24613, 0.41368, -0.28846, -0.097...",6.0,17.5,2.180605
1,ride,ride,1,0.411,0.097780,0.764928,"[0.39289, -0.088632, -0.30353, -0.47467, 0.313...",11.5,2.0,2.095708
4,drive,drive,2,0.379,0.200662,0.737823,"[0.57215, 0.503, 0.068908, -0.41683, 0.081836,...",15.0,3.0,2.075485
6,loud,loud,1,0.443,0.190425,0.466753,"[0.065564, 0.49259, -0.4951, -0.3633, 0.63857,...",7.0,6.0,1.986178


### Wikipedia

In [111]:
text = '''Deep learning (also known as deep structured learning) is part of a broader family 
of machine learning methods based on artificial neural networks with representation 
        learning. Learning can be supervised, semi-supervised or unsupervised.[1][2][3]Deep-learning 
        architectures such as deep neural networks, deep belief networks, recurrent neural networks 
        and convolutional neural networks have been applied to fields including computer vision, 
        machine vision, speech recognition, natural language processing, audio recognition, social 
        network filtering, machine translation, bioinformatics, drug design, medical image analysis, 
        material inspection and board game programs, where they have produced results comparable 
        to and in some cases surpassing human expert performance.[4][5][6]Artificial neural networks 
        (ANNs) were inspired by information processing and distributed communication nodes in biological 
        systems. ANNs have various differences from biological brains. Specifically, neural networks tend 
        to be static and symbolic, while the biological brain of most living organisms is dynamic (plastic) 
        and analogue.[7][8][9]The adjective "deep" in deep learning refers to the use of multiple layers in 
        the network. Early work showed that a linear perceptron cannot be a universal classifier, and then 
        that a network with a nonpolynomial activation function with one hidden layer of unbounded width 
        can on the other hand so be. Deep learning is a modern variation which is concerned with an unbounded
        number of layers of bounded size, which permits practical application and optimized implementation, 
        while retaining theoretical universality under mild conditions. In deep learning the layers are 
        also permitted to be heterogeneous and to deviate widely from biologically informed connectionist 
        models, for the sake of efficiency, trainability and understandability, whence the "structured" part.'''

In [112]:
doc = nlp(text)

257 words in document
152 words in vocabulary
110 words in vocabulary without duplicates


In [113]:
df = df_from_doc(doc)

In [115]:
df = df.sort_values(by=['overall_ranking'], ascending=False).head(50)

In [118]:
predict_outputsize(df)

Word position 0:
Can you translate this word:   nonpolynomial   [y/n]  n
Word position 3:
Can you translate this word:   bioinformatic   [y/n]  n
Word position 7:
Can you translate this word:   neural   [y/n]  n
Word position 15:
Can you translate this word:   supervised   [y/n]  n
Word position 31:
Can you translate this word:   heterogeneous   [y/n]  n


,token,lemma,appearance,difficulty,relative freqency,keyword score,difficulty_rank,keyword_rank,overall_ranking
78,nonpolynomial,nonpolynomial,1,0.889,1.000000,0.348088,1.0,60.5,4.015088
108,trainability,trainability,1,0.734,0.934694,0.349549,3.0,55.0,3.486243
109,understandability,understandability,1,0.722,0.724685,0.349549,5.0,55.0,3.240234
33,bioinformatics,bioinformatic,1,0.738,1.000000,0.000000,2.0,87.5,3.214000
19,convolutional,convolutional,1,0.703,0.489316,0.521135,7.0,34.0,3.119451
82,unbounded,unbounded,2,0.705,0.270206,0.713559,6.0,14.0,3.098765
104,connectionist,connectionist,1,0.723,0.741101,0.000000,4.0,87.5,2.910101
10,neural,neural,6,0.570,0.197697,0.960700,21.0,1.0,2.868397
9,artificial,artificial,1,0.526,0.001449,0.912344,32.0,4.0,2.491793
18,recurrent,recurrent,1,0.612,0.009786,0.564295,14.5,20.5,2.410081
